In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')
sns.set_context('notebook')

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix,classification_report, roc_curve, log_loss, brier_score_loss, roc_auc_score, make_scorer
from timeit import default_timer as timer

In [ ]:
#Cross Validation Function

def CrossVal(model):

    start_cross_val = timer()

    scores = cross_val_score(model, X_train, y_train)

    print('Mean Accuracy:',round(scores.mean()*100,2),'%')
    print('Accuracy Standard Deviation',round(scores.std()*100,2),'%')

    end_cross_val = timer()
    print('Cross Validation Time:', round(end_cross_val - start_cross_val,1), 'seconds')

In [2]:
#murders_full = pd.read_csv('C:\\Users\\Classy\\Desktop\\murders_short.csv')
murders_no_unknowns = pd.read_csv('C:\\Users\\Classy\\Desktop\\murders_no_unknowns.csv')
#murders_unknown_ethnic = pd.read_csv('C:\\Users\\Classy\\Desktop\\murders_no_unknowns_except_VicEthnic.csv')

In [3]:
murders_no_unknowns.Solved.value_counts()

1    220148
0     88813
Name: Solved, dtype: int64

In [4]:
# Balance

solved_count, unsolved_count = murders_no_unknowns.Solved.value_counts()
solved = murders_no_unknowns[murders_no_unknowns.Solved == 1]
unsolved = murders_no_unknowns[murders_no_unknowns.Solved == 0]
solved_sample = solved.sample(unsolved_count, random_state = 33)

murders_no_unknowns = pd.concat([unsolved, solved_sample], axis = 0)

murders_no_unknowns.Solved.value_counts()

1    88813
0    88813
Name: Solved, dtype: int64

In [5]:
murders_no_unknowns.columns

Index(['Agentype', 'Solved', 'Month', 'Homicide', 'Situation', 'VicAge',
       'VicSex', 'VicRace', 'VicEthnic', 'Weapon', 'VicCount', 'MSA',
       'OriClearance', 'OriCases', 'WhiteMurderPercent', 'NonHispanicPercent'],
      dtype='object')

In [6]:
#Full Training Set Random Forest

model = RandomForestClassifier(n_estimators = 1000,
                                  max_depth = 20,
                                  random_state = 33,
                                  n_jobs = 3)

df_categorical = murders_no_unknowns[[
                                    'Agentype',
                                    'Month',
                                    'Homicide',
                                    'Situation',
                                    'VicSex',
                                    'VicRace',
                                    'VicEthnic',
                                    'Weapon'
                                        ]].reset_index(drop = True)

df_numerical = murders_no_unknowns[[
                                    'VicAge',
                                    'VicCount',
                                    'OriCases',
                                    'WhiteMurderPercent'
                                    ]].reset_index(drop = True)
                                                    
y = pd.DataFrame(murders_no_unknowns.Solved).reset_index(drop = True)

In [ ]:
print(type(df_categorical))
print(type(df_numerical))
print(type(y))

In [7]:
start_preprocessing = timer()

#Create dummy columns for categorical features
model_df = pd.get_dummies(df_categorical,
                          drop_first = True)

#Add numerical features to model dataframe
model_df = pd.concat([model_df, df_numerical], axis = 1)

#Scale features
X = pd.DataFrame(StandardScaler().fit_transform(model_df), index = model_df.index, columns = model_df.columns)

#Create validation test set to reserve for final validation
X_train, X_validation, y_train, y_validation = train_test_split(X, y, stratify = y, random_state = 33)

end_preprocessing = timer()
print('Preprocessing Time:', round(end_preprocessing - start_preprocessing,1), 'seconds')

Preprocessing Time: 1.8 seconds


In [ ]:
print(type(X))
print(X_train.shape)
print(X_validation.shape)
print(y_train.shape)
print(type(X_train))
print(type(X_validation))
print(type(y_validation))
print(X_validation[0:5])
print(y_validation[0:5])
print(X_train[0:5])
print(y_train[0:5])

In [ ]:
#Train model
start_training = timer()

model.fit(X_train, np.ravel(y_train))

end_training = timer()
print('Training Time:', round(end_training - start_training,1), 'seconds')

In [ ]:
#Use trained model to predict test results
start_prediction = timer()

y_pred = pd.DataFrame(model.predict(X_validation), index = X_validation.index)
y_pred_prob = pd.DataFrame(model.predict_proba(X_validation)[:,1], index = X_validation.index)

end_prediction = timer()
print('Prediction Time:', round(end_prediction - start_prediction,1), 'seconds')

In [ ]:
print(type(y_pred))
print(type(y_pred_prob))
print(y_pred_prob.head())

In [ ]:
#Output test reports
start_reports = timer()

print('Confusion Matrix:\n', confusion_matrix(y_validation, y_pred))
print('Classification Report:\n', classification_report(y_validation, y_pred))
print('Accuracy:', round((sum(y_pred.values == y_validation.values) / len(y_pred.values) * 100)[0],2), "%")
print('Log Loss:', log_loss(y_validation, y_pred_prob))
print('Brier Score Loss:', brier_score_loss(y_validation, y_pred_prob))

fpr, tpr, thresholds = roc_curve(y_validation, y_pred_prob)

plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

print('ROC AUC Score:', roc_auc_score(y_validation, y_pred_prob))

forest_feature_importance = pd.DataFrame(zip(model_df.columns,model.feature_importances_))
forest_feature_importance.sort_values(1, inplace = True)
forest_feature_importance.reset_index(inplace = True, drop = True)

plt.bar(forest_feature_importance[0][-20:],forest_feature_importance[1][-20:])
plt.xticks(rotation = 'vertical')
plt.title('20 Largest Importances')
plt.show()

plt.bar(forest_feature_importance[0][0:20],forest_feature_importance[1][0:20])
plt.xticks(rotation = 'vertical')
plt.title('20 Smallest Importances')
plt.show()

proba_df = y_validation.copy()
proba_df['Predicted Probability'] = y_pred_prob

#The below bins by Predicted Probability
bins = np.arange(0,1.05,0.05)
labels = ['0.05','0.1','0.15','0.2','0.25','0.3','0.35','0.4','0.45','0.5',
          '0.55','0.6','0.65','0.7','0.75','0.8','0.85','0.9','0.95','1']
proba_df['Bin'] = pd.cut(proba_df['Predicted Probability'], bins = bins, labels = labels)
#proba_df.reset_index(drop = True, inplace = True)
chart_df = proba_df.groupby('Bin')['Solved', 'Predicted Probability'].mean()
chart_df['Count'] = proba_df.groupby('Bin')['Solved'].count()
chart_df.reset_index(inplace = True)
chart_df.columns = ['Bin', 'Actual', 'Predicted', 'Count']

print(chart_df)

plt.bar(chart_df.Bin, chart_df.Count)
plt.xticks(rotation = 45)
plt.title('Frequency of Test Cases by Predicted Clearance Probability')
plt.xlabel('Case Clearance Probability')
plt.ylabel('Number of Cases')
plt.show()

plt.scatter(chart_df.Bin, chart_df.Actual, label = 'Actual', alpha = 1)
plt.scatter(chart_df.Bin, chart_df.Predicted, color = 'r', marker = 'D', label = 'Predicted', alpha = 0.75)
plt.legend()
plt.title('Actual and Predicted Clearance Rate by 5% Bin')
plt.xticks(rotation = 45)
plt.xlabel('Bin')
plt.yticks([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
plt.ylabel('Clearance Rate')
plt.show()

plt.scatter(chart_df.Predicted, chart_df.Actual)
plt.plot([0,1],[0,1], linestyle = '--', color = 'g')
plt.title('Actual vs. Predicted Clearance Rate by 5% bin')
plt.xticks([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],rotation = 45)
plt.xlabel('Predicted')
plt.yticks([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
plt.ylabel('Actual')
plt.show()

plt.bar(chart_df.Bin, chart_df.Actual - chart_df.Predicted,)
plt.title('Actual - Predicted Clearance Rate by 5% Bin')
plt.xticks(rotation = 45)
plt.xlabel('Bin (Range = Value - 5% to Value)')
plt.yticks([-0.05,-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04,0.05])
plt.ylabel('Delta Clearance Rate')
plt.show()

print('Sum of Squared Binned Residuals:', binned_sum_of_squared_residuals(y_validation.values, y_pred_prob))

end_reports = timer()
print('Reporting Time:', round(end_reports - start_reports,1), 'seconds')

In [ ]:
print(y_validation)

In [15]:
#custom loss function

def binned_sum_of_squared_residuals(y_actual, y_predicted):

    df = pd.DataFrame(y_actual)
    df.columns = ['Actual']
    df['Predicted Probability'] = y_predicted

    bins = np.arange(0,1.05,0.05)
    labels = ['0.05','0.1','0.15','0.2','0.25','0.3','0.35','0.4','0.45','0.5',
              '0.55','0.6','0.65','0.7','0.75','0.8','0.85','0.9','0.95','1']
    df['Bin'] = pd.cut(df['Predicted Probability'], bins = bins, labels = labels)
    bin_df = df.groupby('Bin')['Actual', 'Predicted Probability'].mean()
    bin_df.reset_index(inplace = True)
    bin_df.columns = ['Bin', 'Actual', 'Predicted']

    binned_sum_of_squares = sum(((bin_df.Actual - bin_df.Predicted).fillna(0)*100) ** 2)
    
    return binned_sum_of_squares


In [ ]:
print('Binned Sum of Squared Binned Residuals:', binned_sum_of_squared_residuals(y_validation, y_pred_prob))

In [16]:
binned_squared_loss = make_scorer(binned_sum_of_squared_residuals, 
                                  greater_is_better = False,
                                  needs_proba = True)

In [9]:
model = LogisticRegression()

In [22]:
#Add multiple scoring!!
#def cross_val_custom_scoring(model, scoring):

start_cross_val = timer()

scores = cross_val_score(model, X_train, y_train, scoring = binned_squared_loss)

print('Scores:', scores)
print('Mean:', round(scores.mean(), 2))
print('Standard Deviation', round(scores.std(), 2))

end_cross_val = timer()
print('Cross Validation Time:', round(end_cross_val - start_cross_val,1), 'seconds')

C:\Users\Classy\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Classy\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  del sys.path[0]
C:\Users\Classy\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Classy\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  del sys.path[0]
C:\Users\Classy\anaconda3\lib\site-package

Scores: [ -478.81520785  -462.17076923  -615.36729123 -8044.48441743
 -7841.71252911]
Mean: -3488.51
Standard Deviation 3638.11
Cross Validation Time: 4.0 seconds


C:\Users\Classy\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  del sys.path[0]


In [ ]:
#train-test a few model types and use custom scoring function and updated output

In [ ]:
#gridsearch best model (likely Random Forest) using custom scoring function

In [39]:
model  = LogisticRegression()

grid = [{'C': [.000000001, .00000001, .0000001, .000001, .00001]}]

scoring = {'f1' : 'f1',
           'accuracy' : 'accuracy',
           'binned squared loss' : binned_squared_loss}

grid_search = GridSearchCV(model, 
                           param_grid = grid, 
                           scoring = scoring, 
                           n_jobs = 3, 
                           verbose = True, 
                           return_train_score = False, 
                           refit = False)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:    3.7s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=3,
             param_grid=[{'C': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]}],
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring={'accuracy': 'accuracy',
                      'binned squared loss': make_scorer(binned_sum_of_squared_residuals, greater_is_better=False, needs_proba=True),
                      'f1

In [40]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,...,std_test_accuracy,rank_test_accuracy,split0_test_binned squared loss,split1_test_binned squared loss,split2_test_binned squared loss,split3_test_binned squared loss,split4_test_binned squared loss,mean_test_binned squared loss,std_test_binned squared loss,rank_test_binned squared loss
0,0.290134,0.026874,0.058518,0.004246,1e-09,{'C': 1e-09},0.561473,0.564721,0.572477,0.568541,...,0.002140,5,-158.274405,-178.420091,-179.132143,-172.752129,-176.482367,-173.012227,7.694300,4
1,0.310885,0.030446,0.063047,0.004093,1e-08,{'C': 1e-08},0.561473,0.564744,0.572477,0.568541,...,0.002150,4,-158.102790,-178.395828,-178.953055,-172.575624,-176.610265,-172.927512,7.741686,3
2,0.321437,0.029114,0.063089,0.003945,1e-07,{'C': 1e-07},0.561767,0.564236,0.572387,0.568756,...,0.002025,3,-157.811863,-175.736469,-177.059276,-172.007648,-176.303633,-171.783778,7.199281,2
3,0.336754,0.025163,0.058616,0.004228,1e-06,{'C': 1e-06},0.562193,0.564486,0.573393,0.568672,...,0.002266,2,-144.888966,-163.875878,-166.642819,-158.496002,-165.120098,-159.804753,7.947070,1
4,0.343942,0.059157,0.052397,0.008405,1e-05,{'C': 1e-05},0.567077,0.568971,0.577262,0.573049,...,0.001900,1,-1767.270985,-2479.764444,-2532.691853,-1875.967586,-2335.686513,-2198.276276,316.101057,5


In [41]:
pd.DataFrame(grid_search.cv_results_).mean_test_accuracy

0    0.591252
1    0.591275
2    0.591425
3    0.592453
4    0.599059
Name: mean_test_accuracy, dtype: float64

I THINK that the frequency histogram would ideally be more evenly distributed.

In [ ]:
proba_df.head()

In [ ]:
results_df = pd.concat([proba_df,
                        df_categorical,
                        df_numerical],
                       axis = 1)
print(results_df.columns)

In [ ]:
bin_breakdown = results_df.groupby(['Bin','VicRace'])['VicRace'].count()
bin_race_breakdown = bin_breakdown.unstack()
bin_race_breakdown['Total'] = results_df.groupby('Bin')['VicRace'].count()
bin_race_breakdown['Black Percent'] = bin_race_breakdown.Black / bin_race_breakdown.Total
bin_race_breakdown['Native Percent'] = bin_race_breakdown['American Indian or Alaskan Native'] / bin_race_breakdown.Total
bin_race_breakdown['Asian Percent'] = bin_race_breakdown.Asian / bin_race_breakdown.Total
bin_race_breakdown['Native Islander Percent'] = bin_race_breakdown['Native Hawaiian or Pacific Islander'] / bin_race_breakdown.Total
bin_race_breakdown['White Percent'] = bin_race_breakdown.White / bin_race_breakdown.Total
bin_race_breakdown

In [ ]:
plt.bar(bin_race_breakdown.index, bin_race_breakdown['Black Percent'])

In [ ]:
plt.bar(bin_race_breakdown.index, bin_race_breakdown['White Percent'])

In [ ]:
bin_breakdown = results_df.groupby('Bin')['OriCases'].mean()
plt.bar(bin_breakdown.index, bin_breakdown)
plt.xticks(rotation = 45)
plt.title('Predicted Clearance Probability by Total Number of Cases Handled by Investigating Agency')
plt.xlabel('Predicted Clearance Probability')
plt.ylabel('Total Number of Cases Handled by Investigating Agency (Bin Average)')
plt.show()

AWESOME!!!!!

In [ ]:
bin_breakdown = results_df.groupby('Bin')['VicAge'].mean()
plt.bar(bin_breakdown.index, bin_breakdown)

In [ ]:
bin_breakdown = results_df.groupby('Bin')['WhiteMurderPercent'].mean()
plt.bar(bin_breakdown.index, bin_breakdown)

In [ ]:
bin_breakdown = results_df.groupby(['Bin','VicSex'])['VicSex'].count()
bin_sex_breakdown = bin_breakdown.unstack()
bin_sex_breakdown['Total'] = results_df.groupby('Bin')['VicSex'].count()
bin_sex_breakdown['Male Percent'] = bin_sex_breakdown.Male / bin_sex_breakdown.Total
bin_sex_breakdown['Female Percent'] = bin_race_breakdown.Female / bin_race_breakdown.Total
bin_sex_breakdown

Add some logreg lines to the above charts

Maybe try a smaller number of bins and see what happens

Oh! Try validation on an unbalanced (or at least differently balanced) dataset and see what the results look like